In [1]:
import pandas as pd
import json
from os.path import join
import os

In [2]:
def load_json(rfdir, rfname):
    print("loading %s ...", rfname)
    with open(join(rfdir, rfname), "r", encoding="utf-8") as rf:
        data = json.load(rf)
        print("%s loaded", rfname)
        return data

In [3]:
df_catboost_0 = pd.read_csv("../classifier_output/catboost_0/test_pub_prediction.csv")
df_lgbm = pd.read_csv("../classifier_output/lgbm_binary/test_pub_prediction.csv")
df_svm = pd.read_csv("../classifier_output/svm_feat_without_emb/test_pub_prediction.csv")
df_rf = pd.read_csv("../classifier_output/rf/test_pub_prediction.csv")
df_catboost_dbpl_2_oag_clean = pd.read_csv("../classifier_output/catboost_0_dblp_2_clean/test_pub_prediction.csv")
df_lgbm_dblp_2_oag_clean = pd.read_csv("../classifier_output/lgbm_dblp_2_clean/test_pub_prediction.csv")

In [4]:
df_catboost_0.rename(columns={"score": "catboost_0"}, inplace=True)
df_lgbm.rename(columns={"score": "lgbm"}, inplace=True)
df_svm.rename(columns={"score": "svm"}, inplace=True)
df_rf.rename(columns={"score": "rf"}, inplace=True)
df_catboost_dbpl_2_oag_clean.rename(columns={"mean": "catboost_dbpl_2_oag_clean"}, inplace=True)
df_lgbm_dblp_2_oag_clean.rename(columns={"mean": "lgbm_dblp_2_oag_clean"}, inplace=True)

In [7]:
df = df_catboost_0[["pid", "bid"]].copy()

In [9]:
df = pd.concat(
    [
        df,
        df_catboost_0["catboost_0"],
        df_lgbm["lgbm"],
        df_svm["svm"],
        df_rf["rf"],
        df_catboost_dbpl_2_oag_clean["catboost_dbpl_2_oag_clean"],
        df_lgbm_dblp_2_oag_clean["lgbm_dblp_2_oag_clean"],
    ], axis=1)

In [10]:
df["mean"] = df.drop(["pid", "bid"], axis=1).mean(axis=1)

In [11]:
df.head()

,pid,bid,catboost_0,lgbm,svm,rf,catboost_dbpl_2_oag_clean,lgbm_dblp_2_oag_clean,mean
0,621454435aee126c0f1d7065,b0,0.018590,0.022770,0.017920,0.028,0.017844,0.022261,0.021231
1,621454435aee126c0f1d7065,b1,0.005550,0.005622,0.030506,0.010,0.007005,0.005156,0.010640
2,621454435aee126c0f1d7065,b2,0.006211,0.007057,0.029261,0.012,0.007679,0.005679,0.011314
3,621454435aee126c0f1d7065,b3,0.009465,0.006741,0.024240,0.018,0.010402,0.006746,0.012599
4,621454435aee126c0f1d7065,b4,0.012631,0.006983,0.035684,0.030,0.011932,0.007411,0.017440


In [12]:
submit = {}
for pid, _df in df.groupby("pid"):
    _df["bid_int"] = _df["bid"].apply(lambda x: int(x[1:]))
    _df = _df.sort_values("bid_int")
    submit[pid] = _df["mean"].tolist()

In [13]:
submit_example = load_json(f"../dataset/PST", "submission_example_test.json")
count = 0
for key in submit_example.keys():
    if submit.get(key) is None:
        count += 1
        print(f"{key} not found")
        continue
    if len(submit[key]) != len(submit_example[key]):
        count += 1
        print(key, len(submit[key]), len(submit_example[key]))

print(count, count / len(submit))

loading %s ... submission_example_test.json
%s loaded submission_example_test.json
0 0.0


In [14]:
for k, v in submit.items():
    for value in v:
        if value < 0 or 1 < value:
            print(k, value)

In [15]:
os.makedirs("../submit", exist_ok=True)
json.dump(
    submit, open("../submit/ensemble_final.json", "w")
)